In [ ]:
#  1  生成最初特征  data_processed.csv
#  2  增加第一版路网特征    get_road_feature.csv
#  3  增加第二版路网特征    get_road_feature1.csv
#  4  生成一些窗口特征，变化特征，加一些LOC特征  get_last_all_features.csv

In [1]:
import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from utils.DataLoader import SHLDataLoader
# from utils.DataProcesser import DataProcesser
from utils.DataAddProcesser import DataProcesser
from utils.DataMergeProcesser import DataMergeProcesser

/home/jnli/.conda/envs/zhaoyayart/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 一 、得到最初特征&得到补充NORMDIFF特征

def get_general_feature(data_path,rate=None):
    data =SHLDataLoader(data_path,rate)
    data.load_all()
    print('raw data load well ')
    dp_val = DataProcesser(data)
    dp_val.process_pipe()
    dp_val.data.df.to_csv('{}/data_processed_addiction_try.csv'.format(data_path))
    print('general feature generate well')
    # data_processed = pd.read_csv('{}/get_general_feauture.csv'.format(data_path))

print('------------------------  validataion  Hand  --------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
get_general_feature(data_path,0.0001)

print('------------------------  validataion  Hand  --------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
get_general_feature(data_path,0.0001)


print('------------------------- train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
get_general_feature(data_path,0.0001) 

print('-------------------------test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
get_general_feature(data_path,0.0001)

# 验证集2  
print('------------------------ 1  validataion  Bag (1/4) --------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
get_general_feature(data_path)


print('------------------------1  validataion   Hand  (2/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
print(os.listdir(data_path))
get_general_feature(data_path)


# 验证集3 
print('------------------------1  validataion  Hips  (3/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
print(os.listdir(data_path))
get_general_feature(data_path)


# 验证集4
print('------------------------1  validataion  Torso (4/4) -----------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
print(os.listdir(data_path))
get_general_feature(data_path)

print('-------------------------2 test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
get_general_feature(data_path)

# 训练集2
print('-------------------------3  train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
get_general_feature(data_path)


print('-------------------------3  train   Hand   (2/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
print(os.listdir(data_path))
get_general_feature(data_path)



# 训练集3
print('-------------------------3  train   Bag   (3/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
print(os.listdir(data_path))
get_general_feature(data_path)


# 训练集4
print('-------------------------3  train Torso   (4/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
print(os.listdir(data_path))
get_general_feature(data_path)

# 二 、数据集检测
data_path_train =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
data_path_valid = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
data_path_test = '/home/jnli/SHL_2023/SHL2023/data2023/test'

data_processed_train = pd.read_csv('{}/data_processed_train_1.csv'.format(data_path_train))
                                                                        
data_processed_validate = pd.read_csv('{}/data_processed_validate_1.csv'.format(data_path_valid))
# data_processed_test = pd.read_csv('{}/data_processed_test_1.csv'.format(data_path_test))
# 2.1 看一下把100个数据一组，统一成一个标签合理不，发现是合理的，标签压缩后准确率0.999
import numpy as np
df = data_processed_validate
# 计算预测准确的数量
# 计算预测准确的数量
predict_num = df.apply(lambda row: sum(row['label_mode'] == label for label in eval(row['label_all'])), axis=1)
# 计算总数量
correct_num = predict_num.sum()
total_predictions = df['label_num'].sum()

# 计算准确率
accuracy = correct_num / total_predictions
print(f"标签预测的准确率: {accuracy}")
# 2.2 保存预测集的标签
df = data_processed_validate[['time_all','label_mode']]
time2label = {s_time: df.iloc[i]['label_mode'] for i in range(len(df)) for s_time in eval(df.iloc[i]['time_all'])}
# 2.3 查看测试集时间戳情况，看看能不能100个一起预测 (发现测试的时间戳基本是递增的，分布和验证集几乎一样)
data_processed_test = pd.read_csv('{}/data_processed_test_1_discarded.csv'.format(data_path_test))
data_processed_validate = pd.read_csv('{}/data_processed_validate_1_discard.csv'.format(data_path_valid))
df = pd.DataFrame(data_processed_test['time'])
print(len(df ))
print(sum(df['time'].diff()==1000.0))
df1 = pd.DataFrame( data_processed_validate['time']) 
print(len(df1))
print(sum(df1['time'].diff()==1000.0))
# 2.4  看一下保存测试集的方法对不对，拿Bag 验证集看
data_path_valid_1 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
data_processed_validate_1 = pd.read_csv('{}/get_road_feature.csv'.format(data_path_valid_1))
data_processed_validate_1['time_all'] = data_processed_validate_1['time_all'].apply(lambda x: x.strip('[]'))
timestamps = data_processed_validate_1['time_all'].str.split(',').explode().reset_index(drop=True).astype(int)
labels = data_processed_validate_1['label_mode'].repeat(data_processed_validate_1['time_all'].str.split(',').apply(len)).reset_index(drop=True).astype(int)
new_df = pd.DataFrame({'timestamp': timestamps, 'label': labels})
with open('{}/data_detail_validate_1.pickle'.format(data_path_valid_1), 'rb') as file:
    data = pickle.load(file)
data.label
df1 =data.label
df2 = new_df
sum(df1['time']==(df2['timestamp']))
sum(df1['label']==(df2['label']))
14393303/14395941    # 0.999816753903062
# 2.5  确认一下测试集的时间戳有没有错
data_path_test = '/home/jnli/SHL_2023/SHL2023/data2023/test'
data_processed_validate = pd.read_csv('{}/get_road_feature.csv'.format(data_path_test))
label_index  = pd.read_table('/home/jnli/SHL_2023/SHL2023/data2023/test/Label_idx.txt', header = None)
label_index.iloc[:, 0]= label_index.iloc[:, 0].astype('int')
data_processed_validate['time_all'] = data_processed_validate['time_all'].apply(lambda x: x.strip('[]'))
timestamps = data_processed_validate['time_all'].str.split(',').explode().reset_index(drop=True).astype(int)
sum(timestamps == label_index.iloc[:, 0]) #46385816


#  三 、 路网特征NORMDIFF特征拼接,增加LOCATION 特征

In [1]:
import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from utils.DataLoader import SHLDataLoader
# from utils.DataProcesser import DataProcesser
from utils.DataAddProcesser import DataProcesser
from utils.DataMergeProcesser import DataMergeProcesser

/home/jnli/.conda/envs/zhaoyayart/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 验证集2  
print('------------------------ 1  validataion  Bag (1/4) --------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()


print('------------------------1  validataion   Hand  (2/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()



# 验证集3 
print('------------------------1  validataion  Hips  (3/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()



# 验证集4
print('------------------------1  validataion  Torso (4/4) -----------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

------------------------ 1  validataion  Bag (1/4) --------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'Mag.txt', 'data_processed_addiction.csv']
data load well
raw length:  143960
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distance', 'is_stop'}
after get more loc length:  143960
all features saved well
------------------------1  validataion   Hand  (2/4) --------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'Mag.txt', 'data_processed_addiction.csv']
data load well
raw length:  143960
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distanc

In [3]:
print('-------------------------2 test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

-------------------------2 test--------------------------------------
['get_all_features.csv', 'Label_idx.txt', 'Gyr.txt', 'Acc.txt', 'Fighting_zsn_predictions.txt', 'get_road_feature_1.csv', 'Location.txt', 'GPS.txt', 'Mag.txt', 'data_processed_addiction.csv']
data load well
raw length:  463859
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distance', 'is_stop'}
after get more loc length:  463859
all features saved well


In [4]:
# 训练集2
print('-------------------------3  train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

print('-------------------------3  train   Hand   (2/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

# 训练集3
print('-------------------------3  train   Hips  (3/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

# 训练集4
print('-------------------------3  train Torso   (4/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()

-------------------------3  train Bag  (1/4) --------------------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv']
data load well
raw length:  980525
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distance', 'is_stop'}
after get more loc length:  980525
all features saved well
-------------------------3  train   Hand   (2/4) --------------------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv']
data load well
raw length:  980525
------------------------1 Data preparation completed-----------------------
-----------

#  四 、 生成变化特征，窗口特征

import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from utils.DataLoader import SHLDataLoader
# from utils.DataProcesser import DataProcesser
from utils.DataAddProcesser import DataProcesser
from utils.DataMergeProcesser import DataMergeProcesser,get_change_windows_Processer

In [2]:
print('------------------------ 1  validataion  Bag (1/4) --------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

------------------------ 1  validataion  Bag (1/4) --------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'Mag.txt', 'data_processed_addiction.csv']
------------------------1 data load well  ------------------------
raw length:  143960
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------


/home/jnli/SHL_2023/SHL2023/utils/prepare.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_window_' + str(window_size) + '_' + func] = df[column].rolling(window=window_size, center=window_center).mean().astype('float32')
/home/jnli/SHL_2023/SHL2023/utils/prepare.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_window_' + str(window_size) + '_' + func] = df[column].rolling(window=window_size, center=window_center).std().astype('float32')
/home/jnli/SHL_2023/SHL2023/utils/prepare.py:98: Perfor

------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_x_kurtosis_window_20_mean', 'MAG_z_mean_window_20_std', 'speed_window_50_std', 'GYR_z_peak_frequency_window_20_std', 'norm_gyr_dlt_count_above_mean_window_20_mean', 'ACC_y_energy_window_20_std', 'error_window_100_mean', 'distance_subway_change_window_100_median', 'distance_Brighton_change_window_50_std', 'distance_subway_change_window_100_std', 'distance_bus_routes_change_window_20_median', 'distance_parks_change_window_20_std', 'GYR_x_median_window_20_mean', 'norm_gyr_min_window_20_mean', 'MAG_x_entropy_window_20_mean', 'ACC_z_entropy_window_20_std', 'MAG_x_energy_window_20_mean', 'north_speed_window_20_mean', 'norm_acc_peak_frequency_window_20_std', 'distance_Brighton_change_window_100_std', 'ACC_z_mean_window_20_std', 'speed_window_20_std', 'norm_acc_dlt_count_above_mean_window_20_std', 'distance_bus_routes_change_window_100_std', 'ACC_z_std_window_20_std', 'distance_Lond

In [3]:
print('------------------------1  validataion   Hand  (2/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()
# 验证集3 
print('------------------------1  validataion  Hips  (3/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()
# 验证集4
print('------------------------1  validataion  Torso (4/4) -----------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

------------------------1  validataion   Hand  (2/4) --------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'Mag.txt', 'data_processed_addiction.csv']
------------------------1 data load well  ------------------------
raw length:  143960
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_x_kurtosis_window_20_mean', 'MAG_z_mean_window_20_std', 'speed_window_50_std', 'GYR_z_peak_frequency_window_20_std', 'norm_gyr_dlt_count_above_mean_window_20_mean', 'ACC_y_energy_window_20_std', 'error_window_100_mean', 'distance_subway_change_window_100_median', 'distance_Brighton_change_window_50_std', 'distance_subway_change_window_100_std', 'distance_bus_routes_change_window_20_median', 'distance_parks_chan

In [4]:
# 训练集2
print('-------------------------3  train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

print('-------------------------3  train   Hand   (2/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

# 训练集3
print('-------------------------3  train   Hips   (3/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

# 训练集4
print('-------------------------3  train Torso   (4/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

-------------------------3  train Bag  (1/4) --------------------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv']
------------------------1 data load well  ------------------------
raw length:  980525
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_x_kurtosis_window_20_mean', 'MAG_z_mean_window_20_std', 'speed_window_50_std', 'GYR_z_peak_frequency_window_20_std', 'norm_gyr_dlt_count_above_mean_window_20_mean', 'ACC_y_energy_window_20_std', 'error_window_100_mean', 'distance_subway_change_window_100_median', 'distance_Brighton_change_window_50_std', 'distance_subway_change_window_100_std', 'distance_bus_rou

In [5]:
print('-------------------------2 test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
merge_data = get_change_windows_Processer(data_path)
merge_data.get_change_windows_features()

-------------------------2 test--------------------------------------
['get_all_features.csv', 'Label_idx.txt', 'Gyr.txt', 'Acc.txt', 'Fighting_zsn_predictions.txt', 'get_road_feature_1.csv', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv']
------------------------1 data load well  ------------------------
raw length:  463859
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_x_kurtosis_window_20_mean', 'MAG_z_mean_window_20_std', 'speed_window_50_std', 'GYR_z_peak_frequency_window_20_std', 'norm_gyr_dlt_count_above_mean_window_20_mean', 'ACC_y_energy_window_20_std', 'error_window_100_mean', 'distance_subway_change_window_100_median', 'distance_Brighton_change_window_50_std', 'distance_subway_change_window_100_s

# 获得最后的实验特征

In [1]:
import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from utils.DataLoader import SHLDataLoader
# from utils.DataProcesser import DataProcesser
from utils.DataMergeProcesser import DataMergeProcesser

/home/jnli/.conda/envs/zhaoyayart/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('------------------------1  validataion   Hand  (1/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()
print('------------------------1  validataion   Bag (2/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()
# 验证集3 
print('------------------------1  validataion  Hips  (3/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()
# 验证集4
print('------------------------1  validataion  Torso (4/4) -----------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

------------------------1  validataion   Hand  (2/4) --------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv', 'get_experiment_features.csv']
data load well
raw length:  143960
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'distance', 'is_stop', 'speed'}
after get more loc length:  143960
more loc features saved well
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'distance_railways_window_20_std', 'speed_window_20_std', 'speed_change_window_20_std', 'distance_railways_change_window_2

In [3]:
print('-------------------------2 test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

-------------------------2 test--------------------------------------
['get_all_features.csv', 'Label_idx.txt', 'Gyr.txt', 'Acc.txt', 'Fighting_zsn_predictions.txt', 'get_road_feature_1.csv', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv', 'get_experiment_features.csv']
data load well
raw length:  463859
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'distance', 'is_stop', 'speed'}
after get more loc length:  463859
more loc features saved well
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'distance_railways_window_20_std', 'speed_window_20_std', 'speed_change_window_20_std', 'distanc

In [4]:
# 训练集2
print('-------------------------3  train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

print('-------------------------3  train   Hand   (2/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

# 训练集3
print('-------------------------3  train   Hips   (3/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

# 训练集4
print('-------------------------3  train Torso   (4/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

-------------------------3  train Bag  (1/4) --------------------------------------
['get_all_features.csv', 'Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv', 'get_experiment_features.csv']
data load well
raw length:  980525
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'distance', 'is_stop', 'speed'}
after get more loc length:  980525
more loc features saved well
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'distance_railways_window_20_std', 'speed_window_20_std', 'speed_change_window_20_std', 'distance_railways_change_wind

In [1]:
import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from utils.DataLoader import SHLDataLoader
# from utils.DataProcesser import DataProcesser
from utils.DataMergeProcesser import DataMergeProcesser

/home/jnli/.conda/envs/zhaoyayart/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 训练集2
print('-------------------------3  train Bag  (1/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

print('-------------------------3  train   Hand   (2/4) --------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

# 训练集3
print('-------------------------3  train   Hips   (3/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

# 训练集4
print('-------------------------3  train Torso   (4/4)--------------------------------------')
data_path =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

-------------------------3  train Bag  (1/4) --------------------------------------
['Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv', 'get_experiment_features.csv']
data load well
raw length:  980525
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distance'}
after get more loc length:  980525
more loc features saved well
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------
------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_z_cal_range_window_20_std', 'longitude_window_20_std', 'GYR_z_max_window_20_mean', 'norm_mag_median_window_20_mean', 'norm_acc_kurtosis_window_20_me

In [2]:
print('------------------------1  validataion   Hand  (1/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

print('------------------------1  validataion   Bag (2/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()
# 验证集3 
print('------------------------1  validataion  Hips  (3/4) --------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()
# 验证集4
print('------------------------1  validataion  Torso (4/4) -----------------------------')
data_path  = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

------------------------1  validataion   Hand  (1/4) --------------------------
['Gyr.txt', 'Acc.txt', 'get_road_feature_1.csv', 'Label.txt', 'Location.txt', 'GPS.txt', 'data_processed.csv', 'Mag.txt', 'get_road_feature.csv', 'data_processed_addiction.csv', 'get_experiment_features.csv']
data load well
raw length:  143960
------------------------1 Data preparation completed-----------------------
------------------------ 2  generate more loc features ------------------------
New Feature Added: {'speed', 'distance'}
after get more loc length:  143960
more loc features saved well
------------------------2  get changed  features  ------------------------
------------------------3  get  windows features  ------------------------


/home/jnli/SHL_2023/SHL2023/utils/prepare.py:94: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_window_' + str(window_size) + '_' + func] = df[column].rolling(window=window_size, center=window_center).mean().astype('float32')
/home/jnli/SHL_2023/SHL2023/utils/prepare.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column + '_window_' + str(window_size) + '_' + func] = df[column].rolling(window=window_size, center=window_center).std().astype('float32')


------------------------ changed and windows features get well ------------------------
New Feature Added: {'MAG_z_cal_range_window_20_std', 'longitude_window_20_std', 'GYR_z_max_window_20_mean', 'norm_mag_median_window_20_mean', 'norm_acc_kurtosis_window_20_mean', 'MAG_z_max_window_20_mean', 'norm_acc_entropy_window_20_std', 'MAG_x_avg_frequency_window_20_mean', 'ACC_y_skewness_window_20_mean', 'MAG_y_entropy_window_20_std', 'norm_acc_skewness_window_20_std', 'altitude_window_20_mean', 'MAG_y_energy_window_20_mean', 'MAG_x_skewness_window_20_std', 'MAG_x_entropy_window_20_mean', 'MAG_z_peak_frequency_window_20_std', 'accuracy_pct_change', 'MAG_z_skewness_window_20_std', 'MAG_x_energy_window_20_mean', 'MAG_y_avg_frequency_window_20_std', 'distance_railways_change', 'norm_mag_min_window_20_mean', 'ACC_y_std_window_20_mean', 'GYR_z_mean_window_20_mean', 'MAG_z_min_window_20_mean', 'GYR_x_median_window_20_mean', 'GYR_x_entropy_window_20_mean', 'norm_acc_avg_frequency_window_20_mean', 'MAG

In [ ]:
print('-------------------------2 test--------------------------------------')
data_path = '/home/jnli/SHL_2023/SHL2023/data2023/test'
print(os.listdir(data_path))
merge_data = DataMergeProcesser(data_path)
merge_data.get_more_loc_fea()
merge_data.get_change_windows_features()

# 去掉一些效果不好的窗口特征

In [ ]:
import os
import time
import math
import numpy as np
from scipy.fftpack import fft
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
from collections import Counter
from utils.DataLoader import SHLDataLoader
from utils.DataProcesser import DataProcesser

from sklearn.model_selection import train_test_split
from utils.Solver import Solver
import seaborn as sn 
from utils.helper import evaluate
from utils.DataMergeProcesser import DataMergeProcesser
import matplotlib.pyplot as plt
# 一 给数据路径
label_dict = {1: 'Still', 2: 'Walking', 3: 'Run', 4: 'Bike', 5: 'Car', 6: 'Bus', 7: 'Train', 8: 'Subway'}
data_path_train_1 =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
data_path_valid_2 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
data_processed_train_1 = pd.read_csv('{}/get_experiment_features_1.csv'.format(data_path_train_1))
data_processed_validate_2 = pd.read_csv('{}/get_experiment_features_1.csv'.format(data_path_valid_2))
drop_columns =  ['Unnamed: 0',
 'Unnamed: 0.1',
 'label_all',
 'time_all',
 'time',
 'label_mode',
 'label_num',
 'time_series1',
 'time_series2',
 'time_y',
 'time_series3',
 'time_z' ,
 'eid',
 ]
window_columns = ["norm_gyr_median_window_20_mean",
"norm_acc_std_window_20_mean",
"gps_snr_mean_window_20_mean",
"norm_mag_peak_frequency_window_20_mean",
"number_window_20_mean",
"norm_gyr_min_window_20_mean",
"ACC_y_std_window_20_mean",
"GYR_y_std_window_20_mean",
"longitude_window_20_mean",
"longitude_window_20_std"]
windows_discard = []
for i in list(data_processed_train_1.columns):
    if 'window' in i:
        if i not in window_columns:
            windows_discard.append(i)
drop_columns_1 = [
"GYR_y_skewness",
"MAG_y_peak_frequency",
"valid_dlt",
"norm_mag_peak_frequency",
"speed_change",
"MAG_y_kurtosis",
]
drop_columns = drop_columns+windows_discard +drop_columns_1
# 生成最终的数据
data_path_train_1 =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Bag'
data_path_train_2 =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hand'
data_path_train_3 =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Hips'
data_path_train_4 =  '/home/jnli/SHL_2023/SHL2023/data2023/train/Torso'

data_path_valid_1 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Bag'
data_path_valid_2 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hand'
data_path_valid_3 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Hips'
data_path_valid_4 = '/home/jnli/SHL_2023/SHL2023/data2023/validate/Torso'
data_path_test = '/home/jnli/SHL_2023/SHL2023/data2023/test'
def get_last_data(data_path):
    data = pd.read_csv('{}/get_experiment_features_1.csv'.format(data_path))
    data.drop(windows_discard, axis=1, inplace=True) 
    data.to_csv('{}/get_last_features.csv'.format(data_path))
    print('save well')
get_last_data(data_path_valid_1)
get_last_data(data_path_valid_2)
get_last_data(data_path_valid_3)
get_last_data(data_path_valid_4)

get_last_data(data_path_train_1)
get_last_data(data_path_train_2)
get_last_data(data_path_train_3)
get_last_data(data_path_train_4)

get_last_data(data_path_test)